In [1]:
import unittest
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from db.movie_app_db import User, Movie, Mylist, WatchHistory, Genre, Keyword, Studio

class BaseTest(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        # Create an in-memory SQLite engine
        cls.engine = create_engine('sqlite:///:memory:')
        cls.Session = sessionmaker(bind=cls.engine)
        cls.session = cls.Session()

        # Create all tables (Replace this with your actual table definitions)
        User.metadata.create_all(cls.engine)
        Movie.metadata.create_all(cls.engine)
        Mylist.metadata.create_all(cls.engine)
        WatchHistory.metadata.create_all(cls.engine)
        Genre.metadata.create_all(cls.engine)
        Keyword.metadata.create_all(cls.engine)
        Studio.metadata.create_all(cls.engine)

        # Call the service setup method (subclasses can override this)
        cls.setUpService()

    @classmethod
    def tearDownClass(cls):
        # Drop all tables
        User.metadata.drop_all(cls.engine)
        Movie.metadata.drop_all(cls.engine)
        Mylist.metadata.drop_all(cls.engine)
        WatchHistory.metadata.drop_all(cls.engine)
        Genre.metadata.drop_all(cls.engine)
        Keyword.metadata.drop_all(cls.engine)
        Studio.metadata.drop_all(cls.engine)

    @classmethod
    def setUpService(cls):
        """ Placeholder method for setting up services. """
        pass


# Unittest PersonService

In [2]:
from datetime import date
from app.services.person import PersonService


class TestPersonService(BaseTest):
    @classmethod
    def setUpService(cls):
        # Initialize user_service specific to this test case
        cls.person_service = PersonService(cls.session)

    def test_create_person(self):
        self.person_service.create_person(name="JohnSmith", birthdate=date(1995, 7, 25), home_country="USA")
        person = self.person_service.read_person(1)
        self.assertIsNotNone(person)
        self.assertEqual(person.name, "JohnSmith")
        self.assertEqual(person.person_birthdate, date(1995, 7, 25))
        self.assertEqual(person.home_country, "USA")

    def test_read_person(self):
        self.person_service.create_person(name="JaneSmith", birthdate=date(1992, 3, 10), home_country="Canada")
        person = self.person_service.read_person(3)
        self.assertIsNotNone(person)
        self.assertEqual(person.name, "JaneSmith")
        self.assertEqual(person.person_birthdate, date(1992, 3, 10))
        self.assertEqual(person.home_country, "Canada")

    def test_update_person(self):
        self.person_service.create_person(name="OldPersonName", birthdate=date(1990, 12, 15), home_country="UK")
        self.person_service.update_person(4, name="NewPersonName", home_country="Australia")
        person = self.person_service.read_person(4)
        self.assertEqual(person.name, "NewPersonName")
        self.assertEqual(person.home_country, "Australia")

    def test_delete_person(self):
        self.person_service.create_person(name="DeletePerson", birthdate=date(1985, 4, 22), home_country="Germany")
        self.person_service.delete_person(1)
        person = self.person_service.read_person(1)
        self.assertIsNone(person)
        
if __name__ == '__main__':
    # Load all test methods from TestPersonService
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestPersonService)
    
    # Run the test suite
    runner = unittest.TextTestRunner()
    runner.run(suite)

....
----------------------------------------------------------------------
Ran 4 tests in 0.068s

OK


# Unittest UserService

In [11]:
import unittest
from datetime import date
from app.services.user import UserService


class TestUserServic(BaseTest):
    @classmethod
    def setUpService(cls):
        # Initialize user_service specific to this test case
        cls.user_service = UserService(cls.session)
        
    def test_create_user(self):
        self.user_service.create_user(user_name="JohnDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        user = self.user_service.read_user(1)
        self.assertIsNotNone(user)
        self.assertEqual(user.user_name, "JohnDoe")

    def test_read_user(self):
        self.user_service.create_user(user_name="JaneDoe", birthdate=date(1990, 5, 15), password="password", e_mail="jane@example.com")
        user = self.user_service.read_user(3)
        self.assertIsNotNone(user)
        self.assertEqual(user.user_name, "JaneDoe")
        
    def test_read_all_user(self):
        self.user_service.create_user(user_name="JaneDoe2", birthdate=date(1990, 5, 15), password="password", e_mail="jane@example.com")
        user = self.user_service.read_all_users()
        self.assertIsNotNone(user)
        self.assertEqual(user.user_name, "JaneDoe2")

    def test_update_user(self):
        self.user_service.create_user(user_name="OldName", birthdate=date(2000, 1, 1), password="password", e_mail="old@example.com")
        self.user_service.update_user(4, user_name="NewName", e_mail="new@example.com")
        user = self.user_service.read_user(4)
        self.assertEqual(user.user_name, "NewName")
        self.assertEqual(user.e_mail, "new@example.com")

    def test_delete_user(self):
        self.user_service.create_user(user_name="DeleteMe", birthdate=date(2000, 1, 1), password="password", e_mail="delete@example.com")
        self.user_service.delete_user(1)
        user = self.user_service.read_user(1)
        self.assertIsNone(user)
        
if __name__ == '__main__':
    # Load all test methods from TestPersonService
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestUserServic)
    
    # Run the test suite
    runner = unittest.TextTestRunner()
    runner.run(suite)

..EF.
ERROR: test_read_all_user (__main__.TestUserServic.test_read_all_user)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\ElleJan\AppData\Local\Temp\ipykernel_11648\72702379.py", line 28, in test_read_all_user
    self.assertEqual(user.user_name, "JaneDoe2")
  File "c:\App\Anaconda\Lib\unittest\case.py", line 885, in assertEqual
    assertion_func(first, second, msg=msg)
  File "c:\App\Anaconda\Lib\unittest\case.py", line 875, in _baseAssertEqual
    if not first == second:
           ^^^^^^^^^^^^^^^
  File "c:\App\Anaconda\Lib\site-packages\pandas\core\generic.py", line 1577, in __nonzero__
    raise ValueError(
ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

FAIL: test_read_user (__main__.TestUserServic.test_read_user)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\ElleJan\

# Unittest MylistService

In [4]:
 # -------- Mylist CRUD Tests --------
import unittest
from datetime import date
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from app.services.user import UserService
from app.services.mylist import MylistService
from app.services.movie import MovieService
from db.movie_app_db import User, Movie, Mylist, WatchHistory, Genre, Keyword, Studio

class TestMylistService(BaseTest):
    @classmethod
    def setUpService(cls):
        cls.movie_service = MovieService(cls.session)
        cls.user_service = UserService(cls.session)
        cls.mylist_service = MylistService(cls.session)
        
    def test_create_mylist(self):
        mylist = self.mylist_service.read_mylist(1, 1)
        self.assertIsNone(mylist)
        self.user_service.create_user(user_name="JohnDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        self.movie_service.create_movie(movie_name="Inception")
        self.mylist_service.create_mylist(1, 1)
        mylist = self.mylist_service.read_mylist(1, 1)
        self.assertIsNotNone(mylist)
        self.mylist_service.delete_mylist(1, 1)

    def test_read_mylist(self):
        self.user_service.create_user(user_name="JaneDoe", birthdate=date(1990, 5, 15), password="password", e_mail="jane@example.com")
        self.movie_service.create_movie(movie_name="Inception")
        self.mylist_service.create_mylist(1, 1)
        mylist = self.mylist_service.read_mylist(1, 1)
        self.assertIsNotNone(mylist)
        self.mylist_service.delete_mylist(1, 1)

    def test_delete_mylist(self):
        self.user_service.create_user(user_name="PeterDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        self.movie_service.create_movie(movie_name="Inception")
        self.mylist_service.create_mylist(1,1)
        self.mylist_service.delete_mylist(1,1)
        mylist = self.mylist_service.read_mylist(1, 1)
        self.assertIsNone(mylist)
        
if __name__ == '__main__':
    # Load all test methods from TestPersonService
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestMylistService)
    
    # Run the test suite
    runner = unittest.TextTestRunner()
    runner.run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.032s

OK


# Unittest WatchHistoryService

In [5]:
import unittest
from datetime import date
from app.services.user import UserService
from app.services.watchhistory import WatchHistoryService
from app.services.movie import MovieService


class TestWatchHistoryService(BaseTest):
    @classmethod
    def setUpService(cls):
        cls.movie_service = MovieService(cls.session)
        cls.user_service = UserService(cls.session)
        cls.watchhistory_service =WatchHistoryService(cls.session)
        
    def test_create_watch_history(self):
        self.user_service.create_user(user_name="JohnDoe", birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        self.movie_service.create_movie(movie_name="Inception")
        self.watchhistory_service.create_watch_history(user_id=1, movie_id=1, watch_date=date(2024, 9, 11), rating=5.0, is_favorite=True)
        watch_history = self.watchhistory_service.read_watch_history(1, 1)
        self.assertIsNotNone(watch_history)
        self.assertEqual(watch_history.rating, 5.0)

    def test_update_watch_history(self):
        user_name = "JaneDoe"
        self.user_service.create_user(user_name=user_name, birthdate=date(1990, 5, 15), password="password", e_mail="jane@example.com")
        JaneDoe_df = self.user_service.query_by_user_name(search_string=user_name)
        JaneDoe_user_id = JaneDoe_df.iloc[0].user_id
        self.movie_service.create_movie(movie_name="Ariel")
        inception_df = self.movie_service.query_by_title(search_string="Ariel", exact_match=True)
        inception_movie_id = inception_df.iloc[0].movie_id
        self.watchhistory_service.create_watch_history(user_id=JaneDoe_user_id, movie_id=inception_movie_id, watch_date=date(2024, 9, 11))
        self.watchhistory_service.update_watch_history(JaneDoe_user_id, inception_movie_id, rating=4.5, is_favorite=True)
        watch_history = self.watchhistory_service.read_watch_history(JaneDoe_user_id, inception_movie_id)
        self.assertEqual(watch_history.rating, 4.5)
        self.assertTrue(watch_history.is_favorite)

    def test_delete_watch_history(self):
        user_name = "PeterDoe"
        self.user_service.create_user(user_name=user_name, birthdate=date(2000, 1, 1), password="password", e_mail="john@example.com")
        peter_doe_df = self.user_service.query_by_user_name(search_string=user_name)
        peter_doe_user_id = peter_doe_df.iloc[0].user_id
        self.movie_service.create_movie(movie_name="Inception")
        inception_df = self.movie_service.query_by_title(search_string="Inception", exact_match=True)
        inception_movie_id = inception_df.iloc[0].movie_id
        
        self.watchhistory_service.create_watch_history(user_id=peter_doe_user_id, movie_id=inception_movie_id, watch_date=date(2024, 9, 11))
        self.watchhistory_service.delete_watch_history(peter_doe_user_id, inception_movie_id)
        watch_history = self.watchhistory_service.read_watch_history(peter_doe_user_id, inception_movie_id)
        self.assertIsNone(watch_history)
        
if __name__ == '__main__':
    # Load all test methods from TestPersonService
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestWatchHistoryService)
    
    # Run the test suite
    runner = unittest.TextTestRunner()
    runner.run(suite)


...
----------------------------------------------------------------------
Ran 3 tests in 0.039s

OK


# Unittest GenreService

In [6]:

import unittest
from app.services.user import UserService
from app.services.genre import GenreService
from app.services.movie import MovieService


class TestGenreService(BaseTest):
    @classmethod
    def setUpService(cls):
        cls.movie_service = MovieService(cls.session)
        cls.user_service = UserService(cls.session)
        cls.genre_service =GenreService(cls.session)
        
    def test_create_genre(self):
        # given
        genre_name = "XXX"
        
        # when
        self.genre_service.create_genre(genre_name=genre_name)

        # then
        genre_id = self.genre_service.get_id_by_name(genre_name)
        genre = self.genre_service.read_genre(genre_id)    
        self.assertIsNotNone(genre)
        self.assertEqual(genre.genre_name, genre_name)
    
        
    def test_update_genre(self):
        # given
        genre_name = "Adventure"        
        self.genre_service.create_genre(genre_name=genre_name)
        genre_id = self.genre_service.get_id_by_name(genre_name)
        genre = self.genre_service.read_genre(genre_id)    
        
        # when
        new_genre_name = "Sci-Fi"
        self.genre_service.update_genre(genre_id, genre_name=new_genre_name)
        
        
        # then
        genre = self.genre_service.read_genre(genre_id)
        self.assertEqual(genre.genre_name, new_genre_name)
        self.genre_service.delete_genre(genre_id)

    def test_delete_genre(self):
        # given
        genre_name = "Comedy"
        self.genre_service.create_genre(genre_name=genre_name)
        genre_id = self.genre_service.get_id_by_name(genre_name)
        
        # when
        self.genre_service.delete_genre(genre_id)
        
        # then
        genre = self.genre_service.read_genre(genre_id)
        self.assertIsNone(genre)
        
if __name__ == '__main__':
    # Load all test methods from TestPersonService
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestGenreService)
    
    # Run the test suite
    runner = unittest.TextTestRunner()
    runner.run(suite)


...
----------------------------------------------------------------------
Ran 3 tests in 0.029s

OK


# Unittest KeywordService

In [7]:
import unittest
from app.services.keyword import KeywordService

class TestKeywordService(BaseTest):
    @classmethod
    def setUpService(cls):
        cls.keyword_service =KeywordService(cls.session)
        
    def test_create_keyword(self):
        # given
        keyword_name = "Sci-Fi"
        
        # when
        self.keyword_service.create_keyword(keyword_name)
        
        # then
        keyword_id = self.keyword_service.get_id_by_name(keyword_name)
        keyword = self.keyword_service.read_keyword(keyword_id)
        self.assertIsNotNone(keyword)
        self.assertEqual(keyword.keyword_name, keyword_name)
        self.keyword_service.delete_keyword(keyword_id)
            
    def test_update_keyword(self):
        # given
        keyword_name = "Thriller"
        self.keyword_service.create_keyword(keyword_name)
        keyword_id = self.keyword_service.get_id_by_name(keyword_name)
        
        # when
        new_keyword_name = "Mystery"
        self.keyword_service.update_keyword(keyword_id, keyword_name=new_keyword_name)
        
        # then
        keyword = self.keyword_service.read_keyword(keyword_id)
        self.assertEqual(keyword.keyword_name, new_keyword_name)
        self.keyword_service.delete_keyword(keyword_id)

    def test_delete_keyword(self):
        # given
        keyword_name = "Horror"
        self.keyword_service.create_keyword(keyword_name)
        keyword_id = self.keyword_service.get_id_by_name(keyword_name)
        
        # when
        self.keyword_service.delete_keyword(keyword_id)
        
        # then
        keyword = self.keyword_service.read_keyword(keyword_id)
        self.assertIsNone(keyword)
        
if __name__ == '__main__':
    # Load all test methods from TestPersonService
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestKeywordService)
    
    # Run the test suite
    runner = unittest.TextTestRunner()
    runner.run(suite)

.

..
----------------------------------------------------------------------
Ran 3 tests in 0.041s

OK


# Unittest StudioService

In [8]:
import unittest
from app.services.studio import StudioService

class TestStudioService(BaseTest):
    @classmethod
    def setUpService(cls):
        cls.studio_service =StudioService(cls.session)
        
    def test_create_studio(self):
        # given
        studio_name = "Marvel Studios"
        
        # when
        self.studio_service.create_studio(studio_name)
        
        # then
        studio_id = self.studio_service.get_id_by_name(studio_name)
        studio = self.studio_service.read_studio(studio_id)
        self.assertIsNotNone(studio)
        self.assertEqual(studio.studio_name, studio_name)
        self.studio_service.delete_studio(studio_id)

    def test_update_studio(self):
        # given
        studio_name = "Universal"
        self.studio_service.create_studio(studio_name)
        studio_id = self.studio_service.get_id_by_name(studio_name)
        
        # when
        new_studio_name = "Universal Studios"
        self.studio_service.update_studio(studio_id, new_studio_name)
        
        # then
        studio = self.studio_service.read_studio(studio_id)
        self.assertEqual(studio.studio_name, new_studio_name)
        self.studio_service.delete_studio(studio_id)
        
    def test_delete_studio(self):
        # given
        studio_name = "Paramount"
        self.studio_service.create_studio(studio_name)
        studio_id = self.studio_service.get_id_by_name(studio_name)
        
        # when
        self.studio_service.delete_studio(studio_id)
        
        # then
        studio = self.studio_service.read_studio(studio_id)
        self.assertIsNone(studio)

if __name__ == '__main__':
    # Load all test methods from TestPersonService
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestStudioService)
    
    # Run the test suite
    runner = unittest.TextTestRunner()
    runner.run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.086s

OK


# Unittest MovieService

In [9]:
from app.services.movie import MovieService
from app.services.user import UserService
from app.services.person import PersonService
from app.services.genre import GenreService
from app.services.studio import StudioService

import warnings

warnings.filterwarnings('ignore')

class TestMovieService(BaseTest):

    @classmethod
    def setUpService(cls):
        cls.service = MovieService(cls.session)
        cls.user_service = UserService(cls.session)
        cls.person_service = PersonService(cls.session)
        cls.mylist_service = MylistService(cls.session)
        cls.genre_service = GenreService(cls.session)
        cls.keyword_service = KeywordService(cls.session)
        cls.studio_service = StudioService(cls.session)

    def tearDown(self):             
        result_mylists = self.mylist_service.read_all_mylists()        
        if not result_mylists.empty:
            result_mylists.apply(lambda row: self.mylist_service.delete_mylist(row['user_id'], row['movie_id']), axis=1)
               
        results_movies = self.service.query_by_title(search_string="Inception")
        if not results_movies.empty:
            results_movies['movie_id'].apply(self.service.delete_movie)
            
        result_studios = self.studio_service.read_all_studios()        
        if not result_studios.empty:
            result_studios['studio_id'].apply(self.studio_service.delete_studio)
        
        result_users = self.user_service.read_all_users()        
        if not result_users.empty:
            result_users['user_id'].apply(self.user_service.delete_user)            
            
        result_genres = self.genre_service.read_all_genres()        
        if not result_genres.empty:
            result_genres['genre_id'].apply(self.genre_service.delete_genre)
                        
        result_keywords = self.keyword_service.read_all_keywords()        
        if not result_keywords.empty:
            result_keywords['keyword_id'].apply(self.keyword_service.delete_keyword)    
            
        result_persons = self.person_service.read_all_persons()        
        if not result_persons.empty:
            result_persons['person_id'].apply(self.person_service.delete_person)    
              

    # -------- Distinct Values Tests --------
    def test_get_distinct_genre_names(self):
        self.genre_service.create_genre(genre_name="Action")
        self.genre_service.create_genre(genre_name="Drama")
        genres = self.service.get_distinct_genre_names()
        self.assertIn("Action", genres)
        self.assertIn("Drama", genres)

    def test_get_distinct_keyword_names(self):
        self.keyword_service.create_keyword(keyword_name="Sci-Fi")
        self.keyword_service.create_keyword(keyword_name="Action")
        keywords = self.service.get_distinct_keyword_names()
        self.assertIn("Sci-Fi", keywords)
        self.assertIn("Action", keywords)

    def test_get_distinct_person_names(self):
        # Assuming you have a Person model and create_person method
        self.person_service.create_person(name="John Doe")
        self.person_service.create_person(name="Jane Doe")
        persons = self.service.get_distinct_person_names()
        self.assertIn("John Doe", persons)
        self.assertIn("Jane Doe", persons)

    def test_get_distinct_studio_names(self):
        self.studio_service.create_studio(studio_name="Warner Bros")
        self.studio_service.create_studio(studio_name="Paramount")
        studios = self.service.get_distinct_studio_names()
        self.assertIn("Warner Bros", studios)
        self.assertIn("Paramount", studios)

# Run the tests in the notebook
if __name__ == '__main__':
    # Load all test methods from TestPersonService
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestMovieService)
    
    # Run the test suite
    runner = unittest.TextTestRunner()
    runner.run(suite)


EEEE
ERROR: test_get_distinct_genre_names (__main__.TestMovieService.test_get_distinct_genre_names)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\ElleJan\AppData\Local\Temp\ipykernel_11648\2477564737.py", line 36, in tearDown
    result_users = self.user_service.read_all_users()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\App\Python\cinema_app\app\services\user.py", line 48, in read_all_users
    return self.to_dataframe(users).drop(columns =["_sa_instance_state"])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\App\Anaconda\Lib\site-packages\pandas\core\frame.py", line 5581, in drop
    return super().drop(
           ^^^^^^^^^^^^^
  File "c:\App\Anaconda\Lib\site-packages\pandas\core\generic.py", line 4788, in drop
    obj = obj._drop_axis(labels, axis, level=level, errors=errors)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\A